In [3]:
import subprocess
import time
import requests
import socket
import atexit
import subprocess
import time
import socket
import atexit
import argparse
import os
import sys
import json
import requests
from tqdm import tqdm
from torch.utils.data import Dataset
from vllm import LLM, SamplingParams
from transformers import AutoTokenizer

def wait_until_ready(host="localhost", port=8080, timeout=30):
    start = time.time()
    while time.time() - start < timeout:
        try:
            with socket.create_connection((host, port), timeout=1):
                return
        except OSError:
            time.sleep(0.2)
    raise TimeoutError("Server did not start in time.")

def check_proof(proof, split, index):
    """
    Start a dedicated verification server for this (split, index),
    send the proof, and tear down the server after.
    """
    container_name = f"coqstoq-{split}-{index}"
    port = 8080  # optional: dynamically allocate if parallelizing

    # 1. Start dedicated container
    subprocess.run([
        "docker", "run", "--rm", "-d",
        "--name", container_name,
        "-p", f"{port}:8080",
        "coqstoq-full",
        "poetry", "run", "python3",
        "coqstoq/checker_server/server.py", str(split), str(index), "."
    ], check=True)

    # 2. Wait for server to be up
    wait_until_ready(port=port)

    # 3. Send proof via JSON‑RPC
    payload = {
        "jsonrpc": "2.0",
        "method": "check_proof",
        "params": {"proof": proof},
        "id": 1
    }
    try:
        r = requests.post(f"http://localhost:{port}", json=payload, timeout=30)
        r.raise_for_status()
        return r.json()
    finally:
        # 4. Clean up container
        subprocess.run(["docker", "stop", container_name], check=False)


In [8]:
# Argument parsing
parser = argparse.ArgumentParser()
parser.add_argument("--model_name", type=str, default="/home/t-ilshapiro/CoqStoq/fstarcoq-qwq-32b-singleturn-sft") # path that points to the directory with the model name (e.g. fstarcoq-qwq-32b...)
parser.add_argument("--sample_n", type=int, default=1) # how many times we sample for each prompt (i.e. sample on same input)
parser.add_argument("--temperature", type=float, default=0.7)
parser.add_argument("--debug", action="store_true")
parser.add_argument("--num_gpus", type=int, default=2)
args, _ = parser.parse_known_args()

# Load validation data
print("Loading validation data...")
valid_data = []
with open("coq-test-data.jsonl") as file:
    for line in file:
        valid_data.append(json.loads(line))
# if args.debug:
valid_data = valid_data[:10] # can use c. 700 for test benchmark

# Load tokenizer and vLLM engine
print(f"Loading tokenizer and checkpoint from {args.model_name}... ", end="")
tokenizer = AutoTokenizer.from_pretrained(args.model_name)
tokenizer.padding_side = "left"
llm = LLM(model=args.model_name, dtype="bfloat16", max_model_len=16384, tensor_parallel_size=args.num_gpus)

# Prepare prompts
print("Preparing prompts...")
prompts = []
prompt_to_index = []  # (datum_idx, sample_idx)
for datum_idx, datum in enumerate(tqdm(valid_data)):
    prompt = datum["user_prompt"]
    if len(tokenizer(prompt).input_ids) > 8192:
        continue
    for sample_idx in range(args.sample_n):
        prompts.append(prompt)
        prompt_to_index.append((datum_idx,sample_idx))

# Generate with vLLM
print(f"Sampling responses... {args.sample_n} samples per prompt, temp={args.temperature}")
sampling_params = SamplingParams(temperature=args.temperature, max_tokens=16384, n=1)
outputs = llm.generate(prompts, sampling_params)
print("Done sampling")

# Organize responses into valid_data
for datum in valid_data:
    datum["model_generated_response"] = [] # length of this list will be sample_n

for output, (datum_idx, _) in zip(outputs, prompt_to_index):
    response = output.outputs[0].text
    if "<answer>" in response and "</answer>" in response:
        valid_data[datum_idx]["model_generated_response"].append(response) # recall datum_idx is the line number in the jsonl file
        

Loading validation data...
Loading tokenizer and checkpoint from /home/t-ilshapiro/CoqStoq/fstarcoq-qwq-32b-singleturn-sft... INFO 07-07 18:43:03 [config.py:823] This model supports multiple tasks: {'reward', 'score', 'generate', 'embed', 'classify'}. Defaulting to 'generate'.
INFO 07-07 18:43:03 [config.py:1946] Defaulting to use mp for distributed inference
INFO 07-07 18:43:03 [config.py:2195] Chunked prefill is enabled with max_num_batched_tokens=8192.
INFO 07-07 18:43:04 [core.py:455] Waiting for init message from front-end.
INFO 07-07 18:43:04 [core.py:70] Initializing a V1 LLM engine (v0.9.1) with config: model='/home/t-ilshapiro/CoqStoq/fstarcoq-qwq-32b-singleturn-sft', speculative_config=None, tokenizer='/home/t-ilshapiro/CoqStoq/fstarcoq-qwq-32b-singleturn-sft', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config={}, tokenizer_revision=None, trust_remote_code=False, dtype=torch.bfloat16, max_seq_len=16384, download_dir=None, load_format=auto, 

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


WARNING 07-07 18:43:04 [utils.py:2737] Methods determine_num_available_blocks,device_config,get_cache_block_size_bytes,initialize_cache not implemented in <vllm.v1.worker.gpu_worker.Worker object at 0x76481c093f40>
(VllmWorker rank=0 pid=2117613) INFO 07-07 18:43:04 [shm_broadcast.py:289] vLLM message queue communication handle: Handle(local_reader_ranks=[0], buffer_handle=(1, 10485760, 10, 'psm_62c68ad5'), local_subscribe_addr='ipc:///tmp/ea3fcf2e-c324-45ba-b0a5-9c01798afcaa', remote_subscribe_addr=None, remote_addr_ipv6=False)
WARNING 07-07 18:43:04 [utils.py:2737] Methods determine_num_available_blocks,device_config,get_cache_block_size_bytes,initialize_cache not implemented in <vllm.v1.worker.gpu_worker.Worker object at 0x76481c093e50>
(VllmWorker rank=1 pid=2117616) INFO 07-07 18:43:04 [shm_broadcast.py:289] vLLM message queue communication handle: Handle(local_reader_ranks=[0], buffer_handle=(1, 10485760, 10, 'psm_b96f4d34'), local_subscribe_addr='ipc:///tmp/a1acb566-4524-4e49-85

Loading safetensors checkpoint shards:   0% Completed | 0/14 [00:00<?, ?it/s]
Loading safetensors checkpoint shards:   7% Completed | 1/14 [00:00<00:05,  2.24it/s]
Loading safetensors checkpoint shards:  14% Completed | 2/14 [00:00<00:06,  1.99it/s]
Loading safetensors checkpoint shards:  21% Completed | 3/14 [00:01<00:05,  1.89it/s]
Loading safetensors checkpoint shards:  29% Completed | 4/14 [00:02<00:05,  1.81it/s]
Loading safetensors checkpoint shards:  36% Completed | 5/14 [00:02<00:04,  2.16it/s]
Loading safetensors checkpoint shards:  43% Completed | 6/14 [00:02<00:03,  2.11it/s]
Loading safetensors checkpoint shards:  50% Completed | 7/14 [00:03<00:03,  2.04it/s]
Loading safetensors checkpoint shards:  57% Completed | 8/14 [00:04<00:03,  1.95it/s]
Loading safetensors checkpoint shards:  64% Completed | 9/14 [00:04<00:02,  1.89it/s]
Loading safetensors checkpoint shards:  71% Completed | 10/14 [00:05<00:02,  1.84it/s]
Loading safetensors checkpoint shards:  79% Completed | 11/14

(VllmWorker rank=1 pid=2117616) INFO 07-07 18:43:13 [default_loader.py:272] Loading weights took 7.27 seconds


...

(VllmWorker rank=0 pid=2117613) INFO 07-07 18:43:14 [default_loader.py:272] Loading weights took 7.52 seconds
(VllmWorker rank=1 pid=2117616) INFO 07-07 18:43:14 [gpu_model_runner.py:1624] Model loading took 30.7118 GiB and 7.633559 seconds
(VllmWorker rank=0 pid=2117613) INFO 07-07 18:43:14 [gpu_model_runner.py:1624] Model loading took 30.7118 GiB and 7.872171 seconds
(VllmWorker rank=0 pid=2117613) INFO 07-07 18:43:28 [backends.py:462] Using cache directory: /home/t-ilshapiro/.cache/vllm/torch_compile_cache/2c592b1f5b/rank_0_0 for vLLM's torch.compile
(VllmWorker rank=0 pid=2117613) INFO 07-07 18:43:28 [backends.py:472] Dynamo bytecode transform time: 14.10 s
(VllmWorker rank=1 pid=2117616) INFO 07-07 18:43:30 [backends.py:462] Using cache directory: /home/t-ilshapiro/.cache/vllm/torch_compile_cache/2c592b1f5b/rank_1_0 for vLLM's torch.compile
(VllmWorker rank=1 pid=2117616) INFO 07-07 18:43:30 [backends.py:472] Dynamo bytecode transform time: 15.69 s
(VllmWorker rank=0 pid=2117613) 

100%|██████████| 10/10 [00:00<00:00, 57.17it/s]


Sampling responses... 1 samples per prompt, temp=0.7


Processed prompts: 100%|██████████| 7/7 [07:25<00:00, 63.63s/it, est. speed input: 43.11 toks/s, output: 62.71 toks/s] 

Done sampling


In [10]:
import json

output_path = "coq-test-data-with-responses.jsonl"
with open(output_path, "w") as f:
    for obj in valid_data:
        f.write(json.dumps(obj) + "\n")

In [9]:
# Evaluation
pass_n_cnt = [0 for _ in range(args.sample_n)]
results = []
print("Evaluating model outputs...")

for datum in tqdm(valid_data):
    split = datum["split"]
    index = datum["index"]
    prompt = datum["user_prompt"]
    pass_flag = False

    for i, response in enumerate(datum["model_generated_response"]):
        answer = response.split("<answer>")[1].split("</answer>")[0]
        result_datum = check_proof(answer, split, index)["result"] # check_proof gives e.g. {"result": {"score": 1, "messages": []}, "id": 1, "jsonrpc": "2.0"}
        result, errormsg = bool(result_datum["score"]), result_datum["messages"]
        if result:
            pass_flag = True
        pass_n_cnt[i] += 1 if pass_flag else 0

        if args.debug:
            print("SPLIT", split, "INDEX:", index)
            print("Prompt:")
            print(prompt)
            print("Model Output:")
            print(response)
            print("Passed?", result)
            if not result:
                print(errormsg)
            print()
        else:
            results.append({
                "example_name": datum["name"],
                "prompt": prompt,
                "model_output": response,
                "result": result,
                "errormsg": errormsg
            })

print("")
print("Total data:", len(valid_data))
print("Pass@n:", [x / len(valid_data) for x in pass_n_cnt])

Evaluating model outputs...


  0%|          | 0/10 [00:00<?, ?it/s]

191cdcf9a0155eb754f81578a2608b6a20945e76ea3a3ea49b6cbbd38e0bb5c9


  0%|          | 0/10 [00:10<?, ?it/s]


KeyboardInterrupt: 